In [ ]:
## docker build -f c:\NHLStatistics\Dockerfile -t nhlstatistics:latest .
## docker run -p 8501:8501 nhlstatistics:latest

In [2]:
import streamlit as st
import pandas as pd
import numpy as np
# from urllib2 import Request, urlopen

import requests
import pickle
import itertools
import json
from itertools import chain
from io import StringIO
import ast
import datetime

from tqdm.notebook import tqdm, trange
import time

from pandas.io.json import json_normalize

In [2]:
# year = '2019'
# season_type = '02' ## Regular Season = 02 / Playoffs = 03 / Pre-Season = 01
# max_game_ID = 1290 ## 1290 Games/Season

## Collect Information about Each Team

In [3]:
Teams = requests.get(url='https://statsapi.web.nhl.com/api/v1/teams')
Teams = Teams.json()
Teams = pd.concat([pd.DataFrame(Teams['teams'])['id'],
                   pd.DataFrame(Teams['teams'])['locationName'],
                   pd.DataFrame(Teams['teams'])['teamName'],
                   pd.DataFrame(Teams['teams'])['name'],
                   pd.DataFrame(Teams['teams'])['abbreviation'] ,
                   pd.DataFrame(Teams['teams'])['link'],
], axis = 1)

print(Teams)

    id  locationName        teamName                   name abbreviation  \
0    1    New Jersey          Devils      New Jersey Devils          NJD   
1    2      New York       Islanders     New York Islanders          NYI   
2    3      New York         Rangers       New York Rangers          NYR   
3    4  Philadelphia          Flyers    Philadelphia Flyers          PHI   
4    5    Pittsburgh        Penguins    Pittsburgh Penguins          PIT   
5    6        Boston          Bruins          Boston Bruins          BOS   
6    7       Buffalo          Sabres         Buffalo Sabres          BUF   
7    8      Montréal       Canadiens     Montréal Canadiens          MTL   
8    9        Ottawa        Senators        Ottawa Senators          OTT   
9   10       Toronto     Maple Leafs    Toronto Maple Leafs          TOR   
10  12      Carolina      Hurricanes    Carolina Hurricanes          CAR   
11  13       Florida        Panthers       Florida Panthers          FLA   
12  14     T

## Collect the Whole League Roster

In [4]:
FullRoster = pd.DataFrame()

intCounter = 0
for teamLink in tqdm(Teams['link']):
    SourceData = pd.DataFrame.from_dict(requests.get(url='https://statsapi.web.nhl.com' + teamLink + '/roster').json())['roster'].to_dict()
    for Player in SourceData:
        Person = SourceData[Player]['person']
        Position = SourceData[Player]['position']
        
        CurrentPlayer = StringIO("""ID;FullName;Link;PosCode;PosName;PosType;PosAbbreviation;Team
        """ +
                                 str(Person['id']) + ";" +
                                 Person['fullName'] + ";" +
                                 Person['link'] + ";" +
                                 Position['code'] + ";" +
                                 Position['name'] + ";" +
                                 Position['type'] + ";" +
                                 Position['abbreviation'] + ";" +
                                 Teams['name'][intCounter])
        CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
        
        FullRoster = FullRoster.append(CurrentPlayer, ignore_index=True)
    intCounter += 1
    
print(FullRoster)


          ID         FullName                    Link PosCode     PosName  \
0    8470645   Corey Crawford  /api/v1/people/8470645       G      Goalie   
1    8471233     Travis Zajac  /api/v1/people/8471233       C      Center   
2    8474056      P.K. Subban  /api/v1/people/8474056       D  Defenseman   
3    8475151    Kyle Palmieri  /api/v1/people/8475151       R  Right Wing   
4    8475179   Dmitry Kulikov  /api/v1/people/8475179       D  Defenseman   
..       ...              ...                     ...     ...         ...   
763  8478097        Reid Duke  /api/v1/people/8478097       C      Center   
764  8478106     Dylan Sikura  /api/v1/people/8478106       R  Right Wing   
765  8478462      Nicolas Roy  /api/v1/people/8478462       C      Center   
766  8479639    Dylan Coghlan  /api/v1/people/8479639       D  Defenseman   
767  8480727  Zach Whitecloud  /api/v1/people/8480727       D  Defenseman   

        PosType PosAbbreviation                  Team  
0        Goalie   

In [7]:
# PlayerLinks = FullRoster['Link']
# print(PlayerLinks)

## Player Details

In [8]:
PlayerDetails = pd.DataFrame()

intCounter = 0
for playerLink in tqdm(FullRoster['Link']):
    SourceData = pd.DataFrame.from_dict(requests.get(url='https://statsapi.web.nhl.com' + playerLink).json()).to_dict()['people'][0]
    
    try:
        primaryNumber = str(SourceData['primaryNumber'])
    except:
        primaryNumber = "0"
    try:
        birthStateProvince = SourceData['birthStateProvince']
    except: 
        birthStateProvince = "NA"
    
    CurrentPlayer = StringIO("""ID;Position;FullName;Link;FirstName;LastName;PrimaryNumber;BirthDate;CurrentAge;BirthCity;BirthProvState;BirthCountry;Nationality;Height;Weight;Active;AlternateCaptain;Captain;Rookie;ShootsCatches;RosterStatus
    """ +
                    str(SourceData['id']) + ";" +
                    FullRoster['PosAbbreviation'][intCounter] + ";" +
                    SourceData['fullName'] + ";" +
                    SourceData['link'] + ";" +
                    SourceData['firstName'] + ";" +
                    SourceData['lastName'] + ";" +
                    primaryNumber + ";" +
                    SourceData['birthDate'] + ";" +
                    str(SourceData['currentAge']) + ";" +
                    SourceData['birthCity'] + ";" +
                    birthStateProvince + ";" +
                    SourceData['birthCountry'] + ";" +
                    SourceData['nationality'] + ";" +
                    SourceData['height'] + ";" +
                    str(SourceData['weight']) + ";" +
                    str(SourceData['active']) + ";" +
                    str(SourceData['alternateCaptain']) + ";" +
                    str(SourceData['captain']) + ";" +
                    str(SourceData['rookie']) + ";" +
                    SourceData['shootsCatches'] + ";" +
                    SourceData['rosterStatus'])
                             
    CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
    
    PlayerDetails = PlayerDetails.append(CurrentPlayer, ignore_index=True)
    
    intCounter += 1
    
print(PlayerDetails)


          ID Position         FullName                    Link FirstName  \
0    8470645        G   Corey Crawford  /api/v1/people/8470645     Corey   
1    8471233        C     Travis Zajac  /api/v1/people/8471233    Travis   
2    8474056        D      P.K. Subban  /api/v1/people/8474056      P.K.   
3    8475151       RW    Kyle Palmieri  /api/v1/people/8475151      Kyle   
4    8475179        D   Dmitry Kulikov  /api/v1/people/8475179    Dmitry   
..       ...      ...              ...                     ...       ...   
762  8478097        C        Reid Duke  /api/v1/people/8478097      Reid   
763  8478106       RW     Dylan Sikura  /api/v1/people/8478106     Dylan   
764  8478462        C      Nicolas Roy  /api/v1/people/8478462   Nicolas   
765  8479639        D    Dylan Coghlan  /api/v1/people/8479639     Dylan   
766  8480727        D  Zach Whitecloud  /api/v1/people/8480727      Zach   

       LastName  PrimaryNumber   BirthDate  CurrentAge  BirthCity  ...  \
0      Crawf

## Split Skaters and Goalies - Different Statistics

In [9]:
GoalieDetails = PlayerDetails.loc[PlayerDetails['Position'] == 'G']
SkaterDetails = PlayerDetails.loc[PlayerDetails['Position'] != 'G']

GoalieDetails = GoalieDetails.reset_index()
SkaterDetails = SkaterDetails.reset_index()

GoalieRoster = FullRoster.loc[FullRoster['PosCode'] == 'G']
SkaterRoster = FullRoster.loc[FullRoster['PosCode'] != 'G']

SkaterRoster = SkaterRoster.reset_index()
GoalieRoster = GoalieRoster.reset_index()

print(SkaterRoster['ID'])
print(GoalieRoster['ID'])

0      8471233
1      8474056
2      8475151
3      8475179
4      8476850
        ...   
676    8478097
677    8478106
678    8478462
679    8479639
680    8480727
Name: ID, Length: 681, dtype: int64
0     8470645
1     8478406
2     8473575
3     8480382
4     8478048
       ...   
81    8477293
82    8478499
83    8470594
84    8475215
85    8476861
Name: ID, Length: 86, dtype: int64


## Skater Statistics

In [30]:
# SkaterStatistics.to_hdf("SkaterStats.h5", key = "SkaterStatistics")

2020-12-18 09:02:35.127 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [29]:
Season = str(20192020)

SkaterStatistics = pd.DataFrame()

intCounter = 0
for PlayerLink in tqdm(SkaterDetails['Link']):
    SourceData = pd.DataFrame.from_dict(requests.get(url="https://statsapi.web.nhl.com" + PlayerLink + "/stats?stats=statsSingleSeason&season=" + Season).json()).to_dict()['stats'][0]['splits']
    # print(PlayerLink)
    try:
        SD = pd.DataFrame.from_dict(SourceData)['stat']
        SD = SD.to_dict()[0]
        CurrentPlayer = StringIO("""ID;Position;FullName;Team;TOI;A;G;P;PIM;Shots;Games;Hits;PPG;PPP;PPTOI;EVTOI;PIM2;FaceOffPCT;ShtPercent;GWG;OTG;SHG;SHP;SHTOI;Blocks;PlusMinus;Pts;Shifts;TOIperGame;EVTOIperGame;SHTOIperGame;PPTOIperGame
        """ +
                                 str(SkaterRoster['ID'][intCounter]) + ";" +
                                 SkaterRoster['PosAbbreviation'][intCounter] + ";" +
                                 SkaterRoster['FullName'][intCounter] + ";" +
                                 SkaterRoster['Team'][intCounter] + ";" +
                                 str(SD['timeOnIce']) + ";" +
                                 str(SD['assists']) + ";" +
                                 str(SD['goals']) + ";" +
                                 str(SD['points']) + ";" +
                                 str(SD['pim']) + ";" +
                                 str(SD['shots']) + ";" +
                                 str(SD['games']) + ";" +
                                 str(SD['hits']) + ";" +
                                 str(SD['powerPlayGoals']) + ";" +
                                 str(SD['powerPlayPoints']) + ";" +
                                 str(SD['powerPlayTimeOnIce']) + ";" +
                                 str(SD['evenTimeOnIce']) + ";" +
                                 str(SD['penaltyMinutes']) + ";" +
                                 str(SD['faceOffPct']) + ";" +
                                 str(SD['shotPct']) + ";" +
                                 str(SD['gameWinningGoals']) + ";" +
                                 str(SD['overTimeGoals']) + ";" +
                                 str(SD['shortHandedGoals']) + ";" +
                                 str(SD['shortHandedPoints']) + ";" +
                                 str(SD['shortHandedTimeOnIce']) + ";" +
                                 str(SD['blocked']) + ";" +
                                 str(SD['plusMinus']) + ";" +
                                 str(SD['points']) + ";" +
                                 str(SD['shifts']) + ";" +
                                 str(SD['timeOnIcePerGame']) + ";" +
                                 str(SD['evenTimeOnIcePerGame']) + ";" +
                                 str(SD['shortHandedTimeOnIcePerGame']) + ";" +
                                 str(SD['powerPlayTimeOnIcePerGame']))
        
        CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
        
        SkaterStatistics = SkaterStatistics.append(CurrentPlayer, ignore_index=True)
        intCounter += 1
    except:
        print("Missing Stats for Player " + PlayerLink + " " + SkaterRoster['FullName'][intCounter])
        intCounter += 1

print(SkaterStatistics)


Missing Stats for Player /api/v1/people/8474066 Thomas Hickey
Missing Stats for Player /api/v1/people/8480222 Sebastian Aho
Missing Stats for Player /api/v1/people/8481178 Egor Zamula
Missing Stats for Player /api/v1/people/8476191 Kevan Miller
Missing Stats for Player /api/v1/people/8477460 Laurent Dauphin
Missing Stats for Player /api/v1/people/8479990 Josh Brook
Missing Stats for Player /api/v1/people/8475968 Alex Belzile
Missing Stats for Player /api/v1/people/8478454 Noah Juulsen
Missing Stats for Player /api/v1/people/8481582 Nicholas Robertson
Missing Stats for Player /api/v1/people/8476323 Max McCormick
Missing Stats for Player /api/v1/people/8477981 Roland McKeown
Missing Stats for Player /api/v1/people/8479402 Jake Bean
Missing Stats for Player /api/v1/people/8479066 Ryan Lomberg
Missing Stats for Player /api/v1/people/8479597 Chase Priskie
Missing Stats for Player /api/v1/people/8480015 Owen Tippett
Missing Stats for Player /api/v1/people/8481442 Brady Keeper
Missing Stats f

## Goalie Statistics

In [26]:
Season = str(20192020)

GoalieStatistics = pd.DataFrame()

intCounter = 0
for PlayerLink in tqdm(GoalieDetails['Link']):
    SourceData = pd.DataFrame.from_dict(requests.get(url="https://statsapi.web.nhl.com" + PlayerLink + "/stats?stats=statsSingleSeason&season=" + Season).json()).to_dict()['stats'][0]['splits']
    # print(PlayerLink)
    try:
        SD = pd.DataFrame.from_dict(SourceData)['stat']
        SD = SD.to_dict()[0]
        CurrentPlayer = StringIO("""ID;Position;FullName;Team;TOI;OT;SO;T;W;L;Saves;PPSaves;SHSaves;EVSaves;SHShots;EVShots;PPShots;SvPct;GAA;G;GS;ShotsAgainst;GoalsAgainst;TOIperGame;PPSvPct;SHSvPct;EVSvPct
        """ +
                                 str(GoalieRoster['ID'][intCounter]) + ";" +
                                 GoalieRoster['PosAbbreviation'][intCounter] + ";" +
                                 GoalieRoster['FullName'][intCounter] + ";" +
                                 GoalieRoster['Team'][intCounter] + ";" +
                                 str(SD['timeOnIce']) + ";" +
                                 str(SD['ot']) + ";" +
                                 str(SD['shutouts']) + ";" +
                                 str(SD['ties']) + ";" +
                                 str(SD['wins']) + ";" +
                                 str(SD['losses']) + ";" +
                                 str(SD['saves']) + ";" +
                                 str(SD['powerPlaySaves']) + ";" +
                                 str(SD['shortHandedSaves']) + ";" +
                                 str(SD['evenSaves']) + ";" +
                                 str(SD['shortHandedShots']) + ";" +
                                 str(SD['evenShots']) + ";" +
                                 str(SD['powerPlayShots']) + ";" +
                                 str(SD['savePercentage']) + ";" +
                                 str(SD['goalAgainstAverage']) + ";" +
                                 str(SD['games']) + ";" +
                                 str(SD['gamesStarted']) + ";" +
                                 str(SD['shotsAgainst']) + ";" +
                                 str(SD['goalsAgainst']) + ";" +
                                 str(SD['timeOnIcePerGame']) + ";" +
                                 str(SD['powerPlaySavePercentage']) + ";" +
                                 str(SD['shortHandedSavePercentage']) + ";" +
                                 str(SD['evenStrengthSavePercentage']))
        
        CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
        
        GoalieStatistics = GoalieStatistics.append(CurrentPlayer, ignore_index=True)
        intCounter += 1
    except:
        print("Missing Stats for Player " + PlayerLink + " " + GoalieRoster['FullName'][intCounter])
        intCounter += 1

print(GoalieStatistics)


Missing Stats for Player /api/v1/people/8478933 Michael McNiven
Missing Stats for Player /api/v1/people/8479361 Joseph Woll
Missing Stats for Player /api/v1/people/8477475 Philippe Desrosiers
Missing Stats for Player /api/v1/people/8476444 Christopher Gibson
Missing Stats for Player /api/v1/people/8480420 Collin Delia
Missing Stats for Player /api/v1/people/8480947 Kevin Lankinen
Missing Stats for Player /api/v1/people/8478971 Connor Ingram
Missing Stats for Player /api/v1/people/8478024 Ville Husso
Missing Stats for Player /api/v1/people/8481501 Artyom Zagidulin
Missing Stats for Player /api/v1/people/8480112 Hunter Miska
Missing Stats for Player /api/v1/people/8476341 Anton Forsberg
Missing Stats for Player /api/v1/people/8479973 Stuart Skinner
Missing Stats for Player /api/v1/people/8481478 Jake Kielly
Missing Stats for Player /api/v1/people/8479979 Jake Oettinger
Missing Stats for Player /api/v1/people/8477234 Troy Grosenick
Missing Stats for Player /api/v1/people/8476861 Oscar Dan

## Game Statistics

## Complete List of Games

In [10]:
GameList = pd.DataFrame()
# print(Teams['id'])
Team = 1
for Team in tqdm(Teams['id']):
    TeamGames = pd.DataFrame.from_dict(requests.get(url='http://statsapi.web.nhl.com/api/v1/schedule?teamId=' + str(Team) + '&startDate=2019-10-01&endDate=2020-08-01').json()).to_dict()['dates']
    CurrentTeamGames = pd.DataFrame()
    for Game in TeamGames:
        CurrentGame = StringIO("""Link
        """ +
                               str(TeamGames[Game]['games'][0]['link']))

        CurrentGame = pd.read_csv(CurrentGame, sep = ";")
        CurrentTeamGames = CurrentTeamGames.append(CurrentGame, ignore_index = True)

    GameList = GameList.append(CurrentTeamGames, ignore_index = True)
    GameList = GameList.drop_duplicates()
print(GameList)


                                           Link
0             /api/v1/game/2019020013/feed/live
1             /api/v1/game/2019020018/feed/live
2             /api/v1/game/2019020045/feed/live
3             /api/v1/game/2019020049/feed/live
4             /api/v1/game/2019020064/feed/live
...                                         ...
1094          /api/v1/game/2019021067/feed/live
1095          /api/v1/game/2019020057/feed/live
1096          /api/v1/game/2019020399/feed/live
1097          /api/v1/game/2019020600/feed/live
1098          /api/v1/game/2019011012/feed/live

[1099 rows x 1 columns]


## Details of All Games

In [11]:
SeasonDetails = pd.DataFrame()

for CurrentLink in tqdm(GameList['Link']):
    SourceData = pd.DataFrame.from_dict(requests.get(url='http://statsapi.web.nhl.com' + str(CurrentLink.strip(" "))).json()).to_dict()['liveData']['plays']['allPlays']
    GameDetails = pd.DataFrame()
    
    for Play in tqdm(SourceData):
        try:
            Player1Name = dict(Play['players'][0])['player']['fullName']
            Player1ID = dict(Play['players'][0])['player']['id']
        except:
            Player1Name = "" 
            Player1ID = ""
        try:
            Player2Name = dict(Play['players'][1])['player']['fullName']
            Player2ID = dict(Play['players'][1])['player']['id']
        except:
            Player2Name = ""
            Player2ID = ""
        try:
            Player3Name = dict(Play['players'][2])['player']['fullName']
            Player3ID = dict(Play['players'][2])['player']['id']
        except:
            Player3Name = ""
            Player3ID = ""
        try:
            Player4Name = dict(Play['players'][3])['player']['fullName']
            Player4ID = dict(Play['players'][3])['player']['id']
        except:
            Player4Name = ""
            Player4ID = ""

        try:
            x = dict(Play['coordinates'])['x']
            y = dict(Play['coordinates'])['y']
        except:
            x = ""
            y = ""

        CurrentPlay = StringIO("""Event;EventID;Description;P1Name;P1ID;P2Name;P2ID;P3Name;P3ID;P4Name;P4ID;X;Y;Period;TimeRem;PeriodType
        """ +
                               str(Play['result']['event']) + ";" +
                               str(Play['result']['eventTypeId']) + ";" +
                               str(Play['result']['description']) + ";" +
                               str(Player1Name) + ";" +    
                               str(Player1ID) + ";" +
                               str(Player2Name) + ";" +
                               str(Player2ID) + ";" +
                               str(Player3Name) + ";" +
                               str(Player3ID) + ";" +
                               str(Player4Name) + ";" +
                               str(Player4ID) + ";" +
                               str(x) + ";" +
                               str(y) + ";" +
                               str(Play['about']['period']) + ";" +
                               str(Play['about']['periodTimeRemaining']) + ";" +
                               str(Play['about']['periodType']))
        CurrentPlay = pd.read_csv(CurrentPlay, sep = ";")
        
        GameDetails = GameDetails.append(CurrentPlay, ignore_index=True)
    SeasonDetails = SeasonDetails.append(GameDetails, ignore_index=True)

## Break Events into Classes

In [12]:
Goals = SeasonDetails.loc[SeasonDetails['EventID'] == 'GOAL']
Hits = SeasonDetails.loc[SeasonDetails['EventID'] == 'HIT']
Missed_Shots = SeasonDetails.loc[SeasonDetails['EventID'] == 'MISSED_SHOT']
Shots = SeasonDetails.loc[SeasonDetails['EventID'] == 'SHOT']
Penalties = SeasonDetails.loc[SeasonDetails['EventID'] == 'PENALTY']

print(Goals)
print(Hits)
print(Missed_Shots)
print(Shots)
print(Penalties)

print(SeasonDetails.EventID.unique())

               Event EventID  \
68              Goal    GOAL   
88              Goal    GOAL   
133             Goal    GOAL   
146             Goal    GOAL   
176             Goal    GOAL   
...              ...     ...   
347946          Goal    GOAL   
347977          Goal    GOAL   
348145          Goal    GOAL   
348171          Goal    GOAL   
348201          Goal    GOAL   

                                              Description          P1Name  \
68      Nikita Gusev (1) Snap Shot, assists: Jesper Br...    Nikita Gusev   
88      Blake Coleman (1) Snap Shot, assists: Wayne Si...   Blake Coleman   
133     Sami Vatanen (1) Slap Shot, assists: Kyle Palm...    Sami Vatanen   
146     Blake Coleman (2) Wrist Shot, assists: P.K. Su...   Blake Coleman   
176     Dmitry Kulikov (1) Slap Shot, assists: Nikolaj...  Dmitry Kulikov   
...                                                   ...             ...   
347946  Reilly Smith (1) Slap Shot, assists: William K...    Reilly Smith   

In [13]:
MonahanGoals = Goals.loc[Goals['P1Name'] == "Sean Monahan"]
print(MonahanGoals)

               Event EventID  \
4090            Goal    GOAL   
55975           Goal    GOAL   
88276           Goal    GOAL   
118871          Goal    GOAL   
118880          Goal    GOAL   
127411          Goal    GOAL   
199521          Goal    GOAL   
220623          Goal    GOAL   
239753          Goal    GOAL   
254521          Goal    GOAL   
276895          Goal    GOAL   
276943          Goal    GOAL   
280400          Goal    GOAL   
280814          Goal    GOAL   
281692          Goal    GOAL   
283302          Goal    GOAL   
284226          Goal    GOAL   
285451          Goal    GOAL   
285920          Goal    GOAL   
286117          Goal    GOAL   
286427          Goal    GOAL   
286985          Goal    GOAL   
288902          Goal    GOAL   
289809          Goal    GOAL   
291089          Goal    GOAL   

                                              Description        P1Name  \
4090    Sean Monahan (4) Wrist Shot, assists: Noah Han...  Sean Monahan   
55975   Sean Mona

In [19]:
Goals.to_hdf("Goals.h5", key = 'Goals')
Hits.to_hdf("Hits.h5", key = "Hits")
Missed_Shots.to_hdf("Missed.h5", key = "Missed_Shots")
Shots.to_hdf("Shots.h5", key = "Shots")
Penalties.to_hdf("Penalties.h5", key = "Penalties")

SeasonDetails.to_hdf("AllSeasonDetails.h5", key = "SeasonDetails")
SkaterStatistics.to_hdf("SkaterStats.h5", key = "SkaterStatistics")
GoalieStatistics.to_hdf("GoalieStats.h5", key = "GoalieStatistics")
FullRoster.to_hdf("NHLRoster.h5", key = "FullRoster")
PlayerDetails.to_hdf("PlayerDetails.h5", key = "PlayerDetails")

C:\Users\brett\anaconda3\envs\NHLStatistics\lib\site-packages\pandas\core\generic.py:2434: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['Position', 'FullName', 'Link', 'FirstName', 'LastName', 'BirthDate',
       'BirthCity', 'BirthProvState', 'BirthCountry', 'Nationality', 'Height',
       'ShootsCatches', 'RosterStatus'],
      dtype='object')]

  pytables.to_hdf(


In [25]:
LeagueTable = pd.DataFrame()

SourceData = pd.DataFrame.from_dict(requests.get('https://statsapi.web.nhl.com/api/v1/standings').json())
for Division in SourceData['records']:
#     print(Division)
    DivisionName = Division['division']['name']
    Conference = Division['conference']['name']
    for Team in(Division['teamRecords']):
        print(Team)
        TeamName = Team['team']['name']
        TeamWins = Team['leagueRecord']['wins']
        TeamLosses = Team['leagueRecord']['losses']
        TeamOT = Team['leagueRecord']['ot']
        TeamGF = Team['goalsScored']
        TeamPTS = Team['points']
        TeamGA = Team['goalsAgainst']
        TeamDivisionRank = Team['divisionRank']
        TeamConferenceRank = Team['conferenceRank']
        TeamLeagueRank = Team['leagueRank']
        TeamGP = Team['gamesPlayed']
        
        CurrentTeam = StringIO("""Name;Division;Conference;Wins;Losses;Overtime;Goals For;Goals Against;Points;Division Rank;Conference Rank;League Rank;Games Played
        """ +
                               str(Team['team']['name']) + ":" +
                               str(Division['division']['name']) + ";" +
                               str(Division['conference']['name']) + ";" +
                               str(Team['leagueRecord']['wins']) + ";" +
                               str(Player1Name) + ";" +    
                               str(Player1ID) + ";" +
                               str(Player2Name) + ";" +
                               str(Player2ID) + ";" +
                               str(Player3Name) + ";" +
                               str(Player3ID) + ";" +
                               str(Player4Name) + ";" +
                               str(Player4ID) + ";" +
                               str(x) + ";" +
                               str(y) + ";" +
                               str(Play['about']['period']) + ";" +
                               str(Play['about']['periodTimeRemaining']) + ";" +
                               str(Play['about']['periodType']))
        CurrentTeam = pd.read_csv(CurrentTeam, sep = ";")
        
        LeagueTable = LeagueTable.append(CurrentTeam, ignore_index=True)

{'team': {'id': 15, 'name': 'Washington Capitals', 'link': '/api/v1/teams/15'}, 'leagueRecord': {'wins': 41, 'losses': 20, 'ot': 8, 'type': 'league'}, 'regulationWins': 31, 'goalsAgainst': 215, 'goalsScored': 240, 'points': 90, 'divisionRank': '1', 'divisionL10Rank': '5', 'divisionRoadRank': '1', 'divisionHomeRank': '4', 'conferenceRank': '3', 'conferenceL10Rank': '7', 'conferenceRoadRank': '1', 'conferenceHomeRank': '8', 'leagueRank': '5', 'leagueL10Rank': '18', 'leagueRoadRank': '1', 'leagueHomeRank': '12', 'wildCardRank': '0', 'row': 37, 'gamesPlayed': 69, 'streak': {'streakType': 'ot', 'streakNumber': 1, 'streakCode': 'OT1'}, 'pointsPercentage': 0.6521739130434783, 'ppDivisionRank': '1', 'ppConferenceRank': '3', 'ppLeagueRank': '5', 'lastUpdated': '2020-09-28T23:18:38Z'}


NameError: name 'Play' is not defined

# Historical Data About All Players

In [3]:
Seasons = ['20052006','20062007','20072008','20082009',
           '20092010','20102011','20112012','20122013','20132014','20142015','20152016','20162017','20172018',
           '20182019','20192020',]

FullRoster = pd.DataFrame()

intCounter = 0
for Season in tqdm(Seasons):
    print(Season)
#     TeamList = []
#     Teams = pd.DataFrame.from_dict(requests.get(url='https://statsapi.web.nhl.com/api/v1/teams?expand=season=' + Season).json())['teams']
#     for Team in Teams:
#         TeamList.append(Team['link'])
    TeamList = list(range(1,55))
    for Team in tqdm(TeamList):
        try:
            SourceData = pd.DataFrame.from_dict(requests.get(url='https://statsapi.web.nhl.com/api/v1/teams/' + str(Team) + '?expand=team.roster&season=' + str(Season)).json())['teams'][0]['roster']['roster']
            for Player in SourceData:
                CurrentPlayer = StringIO("""ID;FullName;PosAbbreviation;Link;
                """ +
                                         str(Player['person']['id']) + ";" +
                                         Player['person']['fullName'] + ";" +
                                         Player['position']['abbreviation'] + ";" +
                                         Player['person']['link'])
                CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
                
                FullRoster = FullRoster.append(CurrentPlayer, ignore_index=True)
        except:
            print("Missing " + str(Team))
    
print(FullRoster)

20052006


Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51
Missing 52
Missing 53
Missing 54

20062007


Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51
Missing 52
Missing 53
Missing 54

20072008


Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51
Missing 52
Missing 53
Missing 54

20082009


Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51
Missing 52
Missing 53
Missing 54

20092010


...

Missing 11
Missing 27
Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51
Missing 54

20162017


Missing 11
Missing 27
Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51
Missing 54

20172018


Missing 11
Missing 27
Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51

20182019


Missing 11
Missing 27
Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51

20192020


Missing 11
Missing 27
Missing 31
Missing 32
Missing 33
Missing 34
Missing 35
Missing 36
Missing 37
Missing 38
Missing 39
Missing 40
Missing 41
Missing 42
Missing 43
Missing 44
Missing 45
Missing 46
Missing 47
Missing 48
Missing 49
Missing 50
Missing 51


            ID           FullName PosAbbreviation                    Link  \
0      8444919      Tommy Albelin               D  /api/v1/people/8444919   
1      8449654  Alexander Mogilny              RW  /api/v1/people/8449654   
2      8456850           Ken Klee               D  /api/v1/people/8456850   
3      8457191  Vladimir Malakhov               D  /api/v1/people/8457191   
4      8458522  Richard Matvichuk               D  /api/v1/people/8458522   
...        ...                ...             ...                     ...   
15784  8470594  Marc-Andre Fleury               G  /api/v1/people/8470594   
15785  8475215       Robin Lehner               G  /api/v1/people/8475215   
15786  8476343      Garret Sparks               G  /

In [4]:
FRoster = FullRoster.drop_duplicates()
FRoster.to_csv("Full_Roster.csv")
print(FRoster)

FPlayerDetails = pd.DataFrame()

            ID           FullName PosAbbreviation                    Link  \
0      8444919      Tommy Albelin               D  /api/v1/people/8444919   
1      8449654  Alexander Mogilny              RW  /api/v1/people/8449654   
2      8456850           Ken Klee               D  /api/v1/people/8456850   
3      8457191  Vladimir Malakhov               D  /api/v1/people/8457191   
4      8458522  Richard Matvichuk               D  /api/v1/people/8458522   
...        ...                ...             ...                     ...   
15770  8476987      Jake Bischoff               D  /api/v1/people/8476987   
15778  8478434     Keegan Kolesar              RW  /api/v1/people/8478434   
15780  8479748       Gage Quinney               C  /api/v1/people/8479748   
15781  8479980      Nicolas Hague               D  /api/v1/people/8479980   
15782  8479996         Cody Glass               C  /api/v1/people/8479996   

       Unnamed: 4  
0             NaN  
1             NaN  
2             N

In [5]:
intCounter = 0
FPlayerDetails = pd.DataFrame()

for playerLink in tqdm(FRoster['Link']):
    SourceData = pd.DataFrame.from_dict(requests.get(url='https://statsapi.web.nhl.com' + playerLink).json()).to_dict()['people'][0]
    try:
        primaryNumber = str(SourceData['primaryNumber'])
    except:
        primaryNumber = "0"
    try:
        birthStateProvince = SourceData['birthStateProvince']
    except: 
        birthStateProvince = "NA"
       
    CurrentPlayer = StringIO("""ID;Position;FullName;Link;FirstName;LastName;PrimaryNumber;BirthDate;BirthCity;BirthProvState;BirthCountry;Nationality;Height;Weight;Active;Rookie;RosterStatus
    """ +
                    str(SourceData['id']) + ";" +
                    SourceData['primaryPosition']['code'] + ";" +
                    SourceData['fullName'] + ";" +
                    SourceData['link'] + ";" +
                    SourceData['firstName'] + ";" +
                    SourceData['lastName'] + ";" +
                    primaryNumber + ";" +
                    SourceData['birthDate'] + ";" +
                    SourceData['birthCity'] + ";" +
                    birthStateProvince + ";" +
                    SourceData['birthCountry'] + ";" +
                    SourceData['nationality'] + ";" +
                    SourceData['height'] + ";" +
                    str(SourceData['weight']) + ";" +
                    str(SourceData['active']) + ";" +
                    str(SourceData['rookie']) + ";" +
                    SourceData['rosterStatus'])
                             
    CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
    
    FPlayerDetails = FPlayerDetails.append(CurrentPlayer, ignore_index=True)
    
    intCounter += 1
    
print(FPlayerDetails.loc[73])
FPlayerDetails.to_csv("FPlayerDetails.csv")



ID                               8448208
Position                               R
FullName                    Jaromir Jagr
Link              /api/v1/people/8448208
FirstName                        Jaromir
LastName                            Jagr
PrimaryNumber                         68
BirthDate                     1972-02-15
BirthCity                         Kladno
BirthProvState                       NaN
BirthCountry                         CZE
Nationality                          CZE
Height                             6' 3"
Weight                               230
Active                             False
Rookie                             False
RosterStatus                           N
Name: 73, dtype: object


In [6]:
GoalieRoster = FRoster.loc[FRoster['PosAbbreviation'] == 'G']
SkaterRoster = FRoster.loc[FRoster['PosAbbreviation'] != 'G']

SkaterRoster = SkaterRoster.reset_index()
GoalieRoster = GoalieRoster.reset_index()

FGoalieDetails = FPlayerDetails.loc[FPlayerDetails['Position'] == 'G']
FSkaterDetails = FPlayerDetails.loc[FPlayerDetails['Position'] != 'G']

FGoalieDetails = FGoalieDetails.reset_index()
FSkaterDetails = FSkaterDetails.reset_index()

FSkaterDetails.to_csv("FSkaterDetails.csv")

# Player Statistics Structured for Modelling

In [11]:
Seasons = ['20052006','20062007','20072008','20082009',
           '20092010','20102011','20112012','20122013','20132014','20142015','20152016','20162017','20172018',
           '20182019','20192020',]

AllSkates = pd.DataFrame(columns=['ID'])
# intCounter = 0
for Season in tqdm(Seasons):
    intCounter = 0
    SkaterStatistics = pd.DataFrame()
    for PlayerLink in tqdm(SkaterRoster['Link']):
        SourceData = pd.DataFrame.from_dict(requests.get(url="https://statsapi.web.nhl.com" + PlayerLink + "/stats?stats=statsSingleSeason&season=" + Season).json()).to_dict()['stats'][0]['splits']
        # print(PlayerLink)
        try:
            SD = pd.DataFrame.from_dict(SourceData)['stat']
            SD = SD.to_dict()[0]
            CurrentPlayer = StringIO("ID;Position;FullName" + ';TOI' + str(Season) + ';A' + str(Season) + ';G' + str(Season) + ';P' + str(Season) + ';PIM' + str(Season) + ';Shots' + str(Season) + ';Games' + str(Season) + ';Hits' + str(Season) + ';PPG' + str(Season) + ';PPP' + str(Season) + ';PPTOI' + str(Season) + ';EVTOI' + str(Season) + ';PIM2' + str(Season) + ';FaceOffPCT' + str(Season) + ';ShtPercent' + str(Season) + ';GWG' + str(Season) + ';OTG' + str(Season) + ';SHG' + str(Season) + ';SHP' + str(Season) + ';SHTOI' + str(Season) + ';Blocks' + str(Season) + ';PlusMinus' + str(Season) + ';Pts' + str(Season) + ';Shifts' + str(Season) + ';TOIperGame' + str(Season) + ';EVTOIperGame' + str(Season) + ';SHTOIperGame' + str(Season) + ';PPTOIperGame' + str(Season) + """
                                     """ + 
                                     str(SkaterRoster['ID'][intCounter]) + ";" +
                                     SkaterRoster['PosAbbreviation'][intCounter] + ";" +
                                     SkaterRoster['FullName'][intCounter] + ";" +
                                     str(SD['timeOnIce']) + ";" +
                                     str(SD['assists']) + ";" +
                                     str(SD['goals']) + ";" +
                                     str(SD['points']) + ";" +
                                     str(SD['pim']) + ";" +
                                     str(SD['shots']) + ";" +
                                     str(SD['games']) + ";" +
                                     str(SD['hits']) + ";" +
                                     str(SD['powerPlayGoals']) + ";" +
                                     str(SD['powerPlayPoints']) + ";" +
                                     str(SD['powerPlayTimeOnIce']) + ";" +
                                     str(SD['evenTimeOnIce']) + ";" +
                                     str(SD['penaltyMinutes']) + ";" +
                                     str(SD['faceOffPct']) + ";" +
                                     str(SD['shotPct']) + ";" +
                                     str(SD['gameWinningGoals']) + ";" +
                                     str(SD['overTimeGoals']) + ";" +
                                     str(SD['shortHandedGoals']) + ";" +
                                     str(SD['shortHandedPoints']) + ";" +
                                     str(SD['shortHandedTimeOnIce']) + ";" +
                                     str(SD['blocked']) + ";" +
                                     str(SD['plusMinus']) + ";" +
                                     str(SD['points']) + ";" +
                                     str(SD['shifts']) + ";" +
                                     str(SD['timeOnIcePerGame']) + ";" +
                                     str(SD['evenTimeOnIcePerGame']) + ";" +
                                     str(SD['shortHandedTimeOnIcePerGame']) + ";" +
                                     str(SD['powerPlayTimeOnIcePerGame']))
            
            CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
            
            SkaterStatistics = SkaterStatistics.append(CurrentPlayer, ignore_index=True)
            intCounter += 1
        except:
            # print("Missing Stats for Player " + PlayerLink + " " + SkaterRoster['FullName'][intCounter])
            intCounter += 1

    AllSkates = AllSkates.merge(SkaterStatistics, on='ID', how='outer')
print(AllSkates)




           ID Position_x         FullName_x TOI20052006  A20052006  G20052006  \
0     8444919          D      Tommy Albelin      503:20        6.0        0.0   
1     8449654         RW  Alexander Mogilny      549:19       13.0       12.0   
2     8456850          D           Ken Klee     1393:01       12.0        3.0   
3     8457191          D  Vladimir Malakhov      601:14        5.0        4.0   
4     8458522          D  Richard Matvichuk     1129:41       10.0        1.0   
...       ...        ...                ...         ...        ...        ...   
2750  8476987        NaN                NaN         NaN        NaN        NaN   
2751  8478434        NaN                NaN         NaN        NaN        NaN   
2752  8479748        NaN                NaN         NaN        NaN        NaN   
2753  8479980        NaN                NaN         NaN        NaN        NaN   
2754  8479996        NaN                NaN         NaN        NaN        NaN   

      P20052006  PIM20052

# Full Player Statistics - Season-by-Season

In [8]:
Seasons = ['20052006','20062007','20072008','20082009',
           '20092010','20102011','20112012','20122013','20132014','20142015','20152016','20162017','20172018',
           '20182019','20192020',]

AllSkatesStats = pd.DataFrame(columns=['ID'])
# intCounter = 0
for Season in tqdm(Seasons):
    intCounter = 0
    SkaterStatistics = pd.DataFrame()
    for PlayerLink in tqdm(SkaterRoster['Link']):
        SourceData = pd.DataFrame.from_dict(requests.get(url="https://statsapi.web.nhl.com" + PlayerLink + "/stats?stats=statsSingleSeason&season=" + Season).json()).to_dict()['stats'][0]['splits']
        # print(PlayerLink)
        try:
            SD = pd.DataFrame.from_dict(SourceData)['stat']
            SD = SD.to_dict()[0]
            CurrentPlayer = StringIO("""ID;Season;Position;FullName;TOI;A;G;P;PIM;Shots;Games;Hits;PPG;PPP;PPTOI;EVTOI;PIM2;FaceOffPCT;ShtPercent;GWG;OTG;SHG;SHP;SHTOI;Blocks;PlusMinus;Pts;Shifts;TOIperGame;EVTOIperGame;SHTOIperGame;PPTOIperGame
                                     """ + 
                                     str(SkaterRoster['ID'][intCounter]) + ";" +
                                     str(Season) + ";" + 
                                     SkaterRoster['PosAbbreviation'][intCounter] + ";" +
                                     SkaterRoster['FullName'][intCounter] + ";" +
                                     str(SD['timeOnIce']) + ";" +
                                     str(SD['assists']) + ";" +
                                     str(SD['goals']) + ";" +
                                     str(SD['points']) + ";" +
                                     str(SD['pim']) + ";" +
                                     str(SD['shots']) + ";" +
                                     str(SD['games']) + ";" +
                                     str(SD['hits']) + ";" +
                                     str(SD['powerPlayGoals']) + ";" +
                                     str(SD['powerPlayPoints']) + ";" +
                                     str(SD['powerPlayTimeOnIce']) + ";" +
                                     str(SD['evenTimeOnIce']) + ";" +
                                     str(SD['penaltyMinutes']) + ";" +
                                     str(SD['faceOffPct']) + ";" +
                                     str(SD['shotPct']) + ";" +
                                     str(SD['gameWinningGoals']) + ";" +
                                     str(SD['overTimeGoals']) + ";" +
                                     str(SD['shortHandedGoals']) + ";" +
                                     str(SD['shortHandedPoints']) + ";" +
                                     str(SD['shortHandedTimeOnIce']) + ";" +
                                     str(SD['blocked']) + ";" +
                                     str(SD['plusMinus']) + ";" +
                                     str(SD['points']) + ";" +
                                     str(SD['shifts']) + ";" +
                                     str(SD['timeOnIcePerGame']) + ";" +
                                     str(SD['evenTimeOnIcePerGame']) + ";" +
                                     str(SD['shortHandedTimeOnIcePerGame']) + ";" +
                                     str(SD['powerPlayTimeOnIcePerGame']))
            
            CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
            
            SkaterStatistics = SkaterStatistics.append(CurrentPlayer, ignore_index=True)
            intCounter += 1
        except:
            # print("Missing Stats for Player " + PlayerLink + " " + SkaterRoster['FullName'][intCounter])
            intCounter += 1

    AllSkatesStats = AllSkatesStats.append(SkaterStatistics)
print(AllSkatesStats)



          ID      Season Position           FullName      TOI     A     G  \
0    8444919  20052006.0        D      Tommy Albelin   503:20   6.0   0.0   
1    8449654  20052006.0       RW  Alexander Mogilny   549:19  13.0  12.0   
2    8456850  20052006.0        D           Ken Klee  1393:01  12.0   3.0   
3    8457191  20052006.0        D  Vladimir Malakhov   601:14   5.0   4.0   
4    8458522  20052006.0        D  Richard Matvichuk  1129:41  10.0   1.0   
..       ...         ...      ...                ...      ...   ...   ...   
878  8476987  20192020.0        D      Jake Bischoff    62:24   0.0   0.0   
879  8478434  20192020.0       RW     Keegan Kolesar    12:06   0.0   0.0   
880  8479748  20192020.0        C       Gage Quinney    29:08   1.0   0.0   
881  8479980  20192020.0        D      Nicolas Hague   611:24  10.0   1.0   
882  8479996  20192020.0        C         Cody Glass   521:03   7.0   5.0   

        P   PIM  Shots  ...  SHP   SHTOI  Blocks  PlusMinus   Pts  Shifts

In [44]:
ASS = AllSkatesStats
ASS['Season'] = ASS['Season'].apply(float)
ASS.Season = ASS.Season.round(0)
ASS['Season'] = ASS['Season'].apply(int)
print(ASS)

ASS.to_csv("SkaterSeasonStats.csv")

          ID    Season Position           FullName      TOI     A     G     P  \
0    8444919  20052006        D      Tommy Albelin   503:20   6.0   0.0   6.0   
1    8449654  20052006       RW  Alexander Mogilny   549:19  13.0  12.0  25.0   
2    8456850  20052006        D           Ken Klee  1393:01  12.0   3.0  15.0   
3    8457191  20052006        D  Vladimir Malakhov   601:14   5.0   4.0   9.0   
4    8458522  20052006        D  Richard Matvichuk  1129:41  10.0   1.0  11.0   
..       ...       ...      ...                ...      ...   ...   ...   ...   
878  8476987  20192020        D      Jake Bischoff    62:24   0.0   0.0   0.0   
879  8478434  20192020       RW     Keegan Kolesar    12:06   0.0   0.0   0.0   
880  8479748  20192020        C       Gage Quinney    29:08   1.0   0.0   1.0   
881  8479980  20192020        D      Nicolas Hague   611:24  10.0   1.0  11.0   
882  8479996  20192020        C         Cody Glass   521:03   7.0   5.0  12.0   

      PIM  Shots  ...  SHP 

# Goalie Stats - Season by Season

In [11]:
print(GoalieRoster['Link'])

0      /api/v1/people/8455710
1      /api/v1/people/8466339
2      /api/v1/people/8458075
3      /api/v1/people/8468481
4      /api/v1/people/8470584
                ...          
293    /api/v1/people/8479530
294    /api/v1/people/8479979
295    /api/v1/people/8478007
296    /api/v1/people/8480162
297    /api/v1/people/8478039
Name: Link, Length: 298, dtype: object


In [ ]:
Seasons = ['20052006','20062007','20072008','20082009',
           '20092010','20102011','20112012','20122013','20132014','20142015','20152016','20162017','20172018',
           '20182019','20192020',]
# Seasons = ['20192020']

GoalieStatistics = pd.DataFrame()

AllGoalieStats = pd.DataFrame(columns=['ID'])
# intCounter = 0
for Season in tqdm(Seasons):
    intCounter = 0
    GoalieStatistics = pd.DataFrame()
    for PlayerLink in tqdm(GoalieRoster['Link']):
        SourceData = pd.DataFrame.from_dict(requests.get(url="https://statsapi.web.nhl.com" + PlayerLink + "/stats?stats=statsSingleSeason&season=" + Season).json()).to_dict()['stats'][0]['splits']
        # print(SourceData)
        try:
            SD = pd.DataFrame.from_dict(SourceData)
            SD = SD.to_dict()['stat'][0]
            
            CurrentPlayer = StringIO("""ID;Season;Position;FullName;TOI;OT;SO;W;L;Saves;PPSaves;SHSaves;EVSaves;SHShots;EVShots;PPShots;SvPct;GAA;G;GS;ShotsAgainst;GoalsAgainst;TOIperGame;PPSvPct;SHSvPct;EVSvPct
            """ +
                                     str(GoalieRoster['ID'][intCounter]) + ";" +
                                     str(Season) + ";" + 
                                     GoalieRoster['PosAbbreviation'][intCounter] + ";" +
                                     GoalieRoster['FullName'][intCounter] + ";" +
#                                      GoalieRoster['Team'][intCounter] + ";" +
                                     str(SD['timeOnIce']) + ";" +
                                     str(SD['ot']) + ";" +
                                     str(SD['shutouts']) + ";" +
#                                      str(SD['ties']) + ";" +
                                     str(SD['wins']) + ";" +
                                     str(SD['losses']) + ";" +
                                     str(SD['saves']) + ";" +
                                     str(SD['powerPlaySaves']) + ";" +
                                     str(SD['shortHandedSaves']) + ";" +
                                     str(SD['evenSaves']) + ";" +
                                     str(SD['shortHandedShots']) + ";" +
                                     str(SD['evenShots']) + ";" +
                                     str(SD['powerPlayShots']) + ";" +
                                     str(SD['savePercentage']) + ";" +
                                     str(SD['goalAgainstAverage']) + ";" +
                                     str(SD['games']) + ";" +
                                     str(SD['gamesStarted']) + ";" +
                                     str(SD['shotsAgainst']) + ";" +
                                     str(SD['goalsAgainst']) + ";" +
                                     str(SD['timeOnIcePerGame']) + ";" +
                                     str(SD['powerPlaySavePercentage']) + ";" +
                                     str(SD['shortHandedSavePercentage']) + ";" +
                                     str(SD['evenStrengthSavePercentage']))
            
            CurrentPlayer = pd.read_csv(CurrentPlayer, sep = ";")
            
            GoalieStatistics = GoalieStatistics.append(CurrentPlayer, ignore_index=True)
#             print("Found")
            intCounter += 1
        except:
#             print("Missing Stats for Player " + PlayerLink + " " + GoalieRoster['FullName'][intCounter])
            intCounter += 1

    AllGoalieStats = AllGoalieStats.append(GoalieStatistics)
print(AllGoalieStats)



In [ ]:
AGS = AllGoalieStats
AGS['Season'] = AGS['Season'].apply(float)
AGS.Season = AGS.Season.round(0)
AGS['Season'] = AGS['Season'].apply(int)
print(AGS)

AGS.to_csv("GoalieSeasonStats.csv")

In [12]:
AS = pd.DataFrame(AllSkates)

AS = AS.loc[:,~AS.columns.duplicated()]

print(AS)
print(FSkaterDetails)

AS.to_hdf("AllSkaterStats.h5", key = "AllSkaterStatistics")
FSkaterDetails.to_hdf("AllSkaterDetails.h5", key="AllSkaterDetails")
FSkaterDetails.to_csv("FullSkaterDetails.csv")

2020-12-20 18:55:24.367 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


           ID Position_x         FullName_x TOI20052006  A20052006  G20052006  \
0     8444919          D      Tommy Albelin      503:20        6.0        0.0   
1     8449654         RW  Alexander Mogilny      549:19       13.0       12.0   
2     8456850          D           Ken Klee     1393:01       12.0        3.0   
3     8457191          D  Vladimir Malakhov      601:14        5.0        4.0   
4     8458522          D  Richard Matvichuk     1129:41       10.0        1.0   
...       ...        ...                ...         ...        ...        ...   
2750  8476987        NaN                NaN         NaN        NaN        NaN   
2751  8478434        NaN                NaN         NaN        NaN        NaN   
2752  8479748        NaN                NaN         NaN        NaN        NaN   
2753  8479980        NaN                NaN         NaN        NaN        NaN   
2754  8479996        NaN                NaN         NaN        NaN        NaN   

      P20052006  PIM2005200

C:\Users\brett\anaconda3\envs\NHLStatistics\lib\site-packages\pandas\core\generic.py:2434: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['Position_x', 'FullName_x', 'TOI20052006', 'PPTOI20052006',
       'EVTOI20052006', 'SHTOI20052006', 'TOIperGame20052006',
       'EVTOIperGame20052006', 'SHTOIperGame20052006', 'PPTOIperGame20052006',
       ...
       'Position', 'FullName', 'TOI20192020', 'PPTOI20192020', 'EVTOI20192020',
       'SHTOI20192020', 'TOIperGame20192020', 'EVTOIperGame20192020',
       'SHTOIperGame20192020', 'PPTOIperGame20192020'],
      dtype='object', length=126)]

  pytables.to_hdf(
C:\Users\brett\anaconda3\envs\NHLStatistics\lib\site-packages\pandas\core\generic.py:2434: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_val

In [13]:
FullData = FSkaterDetails.merge(AS, on="ID", how="outer")
FullData = FullData.drop_duplicates(subset=['ID'],ignore_index=True)
print(FullData)

FullData = FullData.loc[:,~FullData.columns.duplicated()]


FullData.to_hdf("Complete Skater Data.hdf", "FullSkater")
FullData.to_csv("CompleteSkaterData.csv")

      index       ID Position_x         FullName_x                    Link  \
0         0  8444919          D      Tommy Albelin  /api/v1/people/8444919   
1         1  8449654          R  Alexander Mogilny  /api/v1/people/8449654   
2         2  8456850          D           Ken Klee  /api/v1/people/8456850   
3         3  8457191          D  Vladimir Malakhov  /api/v1/people/8457191   
4         4  8458522          D  Richard Matvichuk  /api/v1/people/8458522   
...     ...      ...        ...                ...                     ...   
2755   3053  8476987          D      Jake Bischoff  /api/v1/people/8476987   
2756   3054  8478434          R     Keegan Kolesar  /api/v1/people/8478434   
2757   3055  8479748          C       Gage Quinney  /api/v1/people/8479748   
2758   3056  8479980          D      Nicolas Hague  /api/v1/people/8479980   
2759   3057  8479996          C         Cody Glass  /api/v1/people/8479996   

      FirstName   LastName  PrimaryNumber   BirthDate      Birt

C:\Users\brett\anaconda3\envs\NHLStatistics\lib\site-packages\pandas\core\generic.py:2434: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['Position_x', 'FullName_x', 'Link', 'FirstName', 'LastName',
       'BirthDate', 'BirthCity', 'BirthProvState', 'BirthCountry',
       'Nationality',
       ...
       'SHTOIperGame20182019', 'PPTOIperGame20182019', 'TOI20192020',
       'PPTOI20192020', 'EVTOI20192020', 'SHTOI20192020', 'TOIperGame20192020',
       'EVTOIperGame20192020', 'SHTOIperGame20192020', 'PPTOIperGame20192020'],
      dtype='object', length=134)]

  pytables.to_hdf(


In [14]:
FullData['BirthDate'] = pd.to_datetime(FullData['BirthDate'], format = '%Y-%m-%d').dt.strftime('%Y').astype(int)

print(FullData['BirthDate'])

Test = 2018-FullData['BirthDate']
print(Test)

0       1964
1       1969
2       1971
3       1968
4       1973
        ... 
2755    1994
2756    1997
2757    1995
2758    1998
2759    1999
Name: BirthDate, Length: 2760, dtype: int32
0       54
1       49
2       47
3       50
4       45
        ..
2755    24
2756    21
2757    23
2758    20
2759    19
Name: BirthDate, Length: 2760, dtype: int32


In [15]:
pd.set_option('display.max_columns', 20)

Details = ['Target','ID','Position','FullName','Link','FirstName','LastName','Number','Age','City','Province','Country','Nationality','Height','Weight']
Stats = ['Assists','Goals','Points','PIMS','Shots','Games','Hits','PPG','PPP','PPTOI','ESTOI','PIM2','FOPct','ShtPct','GWG','OTG','SHG','SHP','SHTOI','Blocks','PlusMinus','Pts2','Shifts','TOIperGame','EVTOIperGame','SHTOIperGame','PPTOIperGame']


# print(FullData.columns[0:15])    #Details
# print(FullData.columns[19:46])   #2005-06 list(range(19,46))
# print(FullData.columns[49:76])   #2006-07 list(range(49,76))
# print(FullData.columns[77:104])  #2007-08 list(range(77,104))
# print(FullData.columns[105:132]) #2008-09 list(range(105,132))
# print(FullData.columns[133:160]) #2009-10 list(range(133,160))
# print(FullData.columns[161:188]) #2010-11 list(range(161,188))
# print(FullData.columns[189:216]) #2011-12 list(range(189,216))
# print(FullData.columns[217:244]) #2012-13 list(range(217,244))
# print(FullData.columns[245:272]) #2013-14
# print(FullData.columns[273:300]) #2014-15
# print(FullData.columns[301:328]) #2015-16
# print(FullData.columns[329:356]) #2016-17
# print(FullData.columns[357:384]) #2017-18
# print(FullData.columns[385:412]) #2018-19
# print(FullData.columns[413:440]) #2019-20

NewColumnNames = (
      pd.Series(Details).tolist() + 
      (pd.Series(Stats) + "LastYear").tolist() + 
      (pd.Series(Stats) + "2Years").tolist() + 
      (pd.Series(Stats) + "3Years").tolist() + 
      (pd.Series(Stats) + "4Years").tolist() + 
      (pd.Series(Stats) + "5Years").tolist()
     )

CompleteDataFrame = pd.DataFrame(columns = NewColumnNames)


Data2018 = FullData.iloc[:,(list(range(387,388)) + list(range(1,15)) + 
      list(range(357,384)) + list(range(329,356)) +
      list(range(301,328)) + list(range(273,300)) + 
      list(range(245,272)))]
Data2018['BirthDate'] = 2018-Data2018['BirthDate']

Data2017 = FullData.iloc[:,(list(range(359,360)) + list(range(1,15)) + 
      list(range(329,356)) + list(range(301,328)) + 
      list(range(273,300)) + list(range(245,272)) + 
      list(range(217,244)))]
Data2017['BirthDate'] = 2017-Data2017['BirthDate']

Data2016 = FullData.iloc[:,(list(range(331,332)) + list(range(1,15)) + 
      list(range(301,328)) + list(range(273,300)) + 
      list(range(245,272)) + list(range(217,244)) +
      list(range(189,216)))]
Data2016['BirthDate'] = 2016-Data2016['BirthDate']

Data2015 = FullData.iloc[:,(list(range(303,304)) + list(range(1,15)) + 
      list(range(273,300)) + list(range(245,272)) + 
      list(range(217,244)) + list(range(189,216)) +
      list(range(161,188)))]
Data2015['BirthDate'] = 2015-Data2015['BirthDate']

Data2014 = FullData.iloc[:,(list(range(275,276)) + list(range(1,15)) + 
      list(range(245,272)) + list(range(217,244)) + 
      list(range(189,216)) + list(range(161,188)) +
      list(range(133,160)))]
Data2014['BirthDate'] = 2014-Data2014['BirthDate']

Data2013 = FullData.iloc[:,(list(range(275,276)) + list(range(1,15)) + 
      list(range(217,244)) + list(range(189,216)) +
      list(range(161,188)) + list(range(133,160)) +
      list(range(105,132)))]
Data2013['BirthDate'] = 2013-Data2013['BirthDate']

Data2012 = FullData.iloc[:,(list(range(275,276)) + list(range(1,15)) + 
      list(range(189,216)) + list(range(161,188)) + 
      list(range(133,160)) + list(range(105,132)) +
      list(range(77,104)))]
Data2012['BirthDate'] = 2012-Data2012['BirthDate']

Data2011 = FullData.iloc[:,(list(range(275,276)) + list(range(1,15)) + 
      list(range(161,188)) + list(range(133,160)) +
      list(range(105,132)) + list(range(77,104)) + 
      list(range(49,76)))]
Data2011['BirthDate'] = 2011-Data2011['BirthDate']

Data2010 = FullData.iloc[:,(list(range(275,276)) + list(range(1,15)) + 
      list(range(133,160)) + list(range(105,132)) +
      list(range(77,104)) + list(range(49,76)) +
      list(range(19,46)))]
Data2010['BirthDate'] = 2010-Data2010['BirthDate']

###### Convert BirthDate to Year, Subtract from Season Start - AGE
###### Convert Height to cm

print("2018")
Data2018.columns = NewColumnNames
Data2018 = Data2018[Data2018['Target'].notna()].reset_index(drop = True)
print(Data2018)
print("2017")
Data2017.columns = NewColumnNames
Data2017 = Data2017[Data2017['Target'].notna()].reset_index(drop = True)
print(Data2017)
print("2016")
Data2016.columns = NewColumnNames
Data2016 = Data2016[Data2016['Target'].notna()].reset_index(drop = True)
print(Data2016)
print("2015")
Data2015.columns = NewColumnNames
Data2015 = Data2015[Data2015['Target'].notna()].reset_index(drop = True)
print(Data2015)
print("")
Data2014.columns = NewColumnNames
Data2014 = Data2014[Data2014['Target'].notna()].reset_index(drop = True)
print(Data2014)
print("")
Data2013.columns = NewColumnNames
Data2013 = Data2013[Data2013['Target'].notna()].reset_index(drop = True)
print(Data2013)
print("")
Data2012.columns = NewColumnNames
Data2012 = Data2012[Data2012['Target'].notna()].reset_index(drop = True)
print(Data2012)
print("")
Data2011.columns = NewColumnNames
Data2011 = Data2011[Data2011['Target'].notna()].reset_index(drop = True)
print(Data2011)
print("")
Data2010.columns = NewColumnNames
Data2010 = Data2010[Data2010['Target'].notna()].reset_index(drop = True)
print(Data2010)


2018
     Target       ID Position             FullName                    Link  \
0      61.0  8470610        L          Zach Parise  /api/v1/people/8470610   
1      33.0  8470604        C          Jeff Carter  /api/v1/people/8470604   
2       9.0  8468498        D         Brooks Orpik  /api/v1/people/8468498   
3     100.0  8471675        C        Sidney Crosby  /api/v1/people/8471675   
4      51.0  8466138        C         Joe Thornton  /api/v1/people/8466138   
..      ...      ...      ...                  ...                     ...   
901     1.0  8480005        L  Kristian Vesalainen  /api/v1/people/8480005   
902     1.0  8478047        L      Michael Bunting  /api/v1/people/8478047   
903    18.0  8478856        R        Conor Garland  /api/v1/people/8478856   
904     4.0  8480950        D      Ilya Lyubushkin  /api/v1/people/8480950   
905     1.0  8481486        D        Jimmy Schuldt  /api/v1/people/8481486   

    FirstName    LastName  Number  Age           City  ...

     Target       ID Position           FullName                    Link  \
0      34.0  8460542        C       Patrik Elias  /api/v1/people/8460542   
1      34.0  8467351        C        Scott Gomez  /api/v1/people/8467351   
2      35.0  8467407        R       Brian Gionta  /api/v1/people/8467407   
3      20.0  8468542        D        Paul Martin  /api/v1/people/8468542   
4      62.0  8470610        L        Zach Parise  /api/v1/people/8470610   
..      ...      ...      ...                ...                     ...   
877     1.0  8475996        C  Brendan Shinnimin  /api/v1/people/8475996   
878    21.0  8476356        C      Tobias Rieder  /api/v1/people/8476356   
879     0.0  8476868        C  Henrik Samuelsson  /api/v1/people/8476868   
880     1.0  8476921        L   Jordan Martinook  /api/v1/people/8476921   
881     0.0  8477715        C       Tyler Gaudet  /api/v1/people/8477715   

    FirstName    LastName  Number  Age         City  ... SHP5Years  \
0      Patrik    

<ipython-input-15-e2cc2bcb5f93>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data2018['BirthDate'] = 2018-Data2018['BirthDate']
<ipython-input-15-e2cc2bcb5f93>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data2017['BirthDate'] = 2017-Data2017['BirthDate']
<ipython-input-15-e2cc2bcb5f93>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [16]:
pd.set_option('display.max_columns', None)

CompleteDataFrame = pd.DataFrame(columns = NewColumnNames)

CompleteDataFrame = CompleteDataFrame.append([Data2010,Data2011,Data2012,Data2013,Data2014,Data2015,Data2016,Data2017,Data2018])

CompleteDataFrame = CompleteDataFrame.drop(columns = ['ID', 'FullName', 'Link', 'FirstName', 'LastName', 'Number', 'City','Province','Nationality'])


print(CompleteDataFrame)


     Target Position Age Country  Height Weight  AssistsLastYear  \
0      34.0        C  34     CZE   6' 1"    190             29.0   
1      34.0        C  31     USA  5' 11"    200             47.0   
2      35.0        R  31     USA   5' 7"    178             18.0   
3      20.0        D  29     USA   6' 1"    200              9.0   
4      62.0        L  26     USA  5' 11"    195             44.0   
..      ...      ...  ..     ...     ...    ...              ...   
901     1.0        L  19     FIN   6' 3"    207              NaN   
902     1.0        L  23     CAN  5' 11"    197              NaN   
903    18.0        R  22     USA  5' 10"    165              NaN   
904     4.0        D  24     RUS   6' 2"    201              NaN   
905     1.0        D  23     USA   6' 0"    203              NaN   

     GoalsLastYear  PointsLastYear  PIMSLastYear  ShotsLastYear  \
0             19.0            48.0          40.0          145.0   
1             12.0            59.0          60.0 

In [17]:
pd.set_option("max_rows", 50)
CDF = CompleteDataFrame
CDF = CDF.fillna(0)
# CompleteDataFrame.dtypes

# pd.to_datetime(FullData['BirthDate'], format = '%Y-%m-%d').dt.strftime('%Y').astype(int)

CDF = CDF.replace({":":"."}, regex=True)
# print(test)
CDF['PPTOILastYear'] = CDF['PPTOILastYear'].astype(float)
CDF['ESTOILastYear'] = CDF['ESTOILastYear'].astype(float)
CDF['SHTOILastYear'] = CDF['SHTOILastYear'].astype(float)
CDF['TOIperGameLastYear'] = CDF['TOIperGameLastYear'].astype(float)
CDF['EVTOIperGameLastYear'] = CDF['EVTOIperGameLastYear'].astype(float)
CDF['SHTOIperGameLastYear'] = CDF['SHTOIperGameLastYear'].astype(float)
CDF['PPTOIperGameLastYear'] = CDF['PPTOIperGameLastYear'].astype(float)
CDF['PPTOI2Years'] = CDF['PPTOI2Years'].astype(float)
CDF['ESTOI2Years'] = CDF['ESTOI2Years'].astype(float)
CDF['SHTOI2Years'] = CDF['SHTOI2Years'].astype(float)
CDF['TOIperGame2Years'] = CDF['TOIperGame2Years'].astype(float)
CDF['EVTOIperGame2Years'] = CDF['EVTOIperGame2Years'].astype(float)
CDF['SHTOIperGame2Years'] = CDF['SHTOIperGame2Years'].astype(float)
CDF['PPTOIperGame2Years'] = CDF['PPTOIperGame2Years'].astype(float)
CDF['PPTOI3Years'] = CDF['PPTOI3Years'].astype(float)
CDF['ESTOI3Years'] = CDF['ESTOI3Years'].astype(float)
CDF['SHTOI3Years'] = CDF['SHTOI3Years'].astype(float)
CDF['TOIperGame3Years'] = CDF['TOIperGame3Years'].astype(float)
CDF['EVTOIperGame3Years'] = CDF['EVTOIperGame3Years'].astype(float)
CDF['SHTOIperGame3Years'] = CDF['SHTOIperGame3Years'].astype(float)
CDF['PPTOIperGame3Years'] = CDF['PPTOIperGame3Years'].astype(float)
CDF['PPTOI4Years'] = CDF['PPTOI4Years'].astype(float)
CDF['ESTOI4Years'] = CDF['ESTOI4Years'].astype(float)
CDF['SHTOI4Years'] = CDF['SHTOI4Years'].astype(float)
CDF['TOIperGame4Years'] = CDF['TOIperGame4Years'].astype(float)
CDF['EVTOIperGame4Years'] = CDF['EVTOIperGame4Years'].astype(float)
CDF['SHTOIperGame4Years'] = CDF['SHTOIperGame4Years'].astype(float)
CDF['PPTOIperGame4Years'] = CDF['PPTOIperGame4Years'].astype(float)
CDF['PPTOI5Years'] = CDF['PPTOI5Years'].astype(float)
CDF['ESTOI5Years'] = CDF['ESTOI5Years'].astype(float)
CDF['SHTOI5Years'] = CDF['SHTOI5Years'].astype(float)
CDF['TOIperGame5Years'] = CDF['TOIperGame5Years'].astype(float)
CDF['EVTOIperGame5Years'] = CDF['EVTOIperGame5Years'].astype(float)
CDF['SHTOIperGame5Years'] = CDF['SHTOIperGame5Years'].astype(float)
CDF['PPTOIperGame5Years'] = CDF['PPTOIperGame5Years'].astype(float)

CDF.dtypes

Target                float64
Position               object
Age                     int64
Country                object
Height                 object
                       ...   
Shifts5Years          float64
TOIperGame5Years      float64
EVTOIperGame5Years    float64
SHTOIperGame5Years    float64
PPTOIperGame5Years    float64
Length: 141, dtype: object

In [18]:
CDF2 = CDF

def parse_ht(ht):
    # format: 7' 0.0"
    ht_ = ht.split("' ")
    ft_ = float(ht_[0])
    in_ = float(ht_[1].replace("\"",""))
    return (12*ft_) + in_

CDF2['Height'] = (CDF2["Height"].apply(lambda x:parse_ht(x)))

CDF2

,Target,Position,Age,Country,Height,Weight,AssistsLastYear,GoalsLastYear,PointsLastYear,PIMSLastYear,ShotsLastYear,GamesLastYear,HitsLastYear,PPGLastYear,PPPLastYear,PPTOILastYear,ESTOILastYear,PIM2LastYear,FOPctLastYear,ShtPctLastYear,GWGLastYear,OTGLastYear,SHGLastYear,SHPLastYear,SHTOILastYear,BlocksLastYear,PlusMinusLastYear,Pts2LastYear,ShiftsLastYear,TOIperGameLastYear,EVTOIperGameLastYear,SHTOIperGameLastYear,PPTOIperGameLastYear,Assists2Years,Goals2Years,Points2Years,PIMS2Years,Shots2Years,Games2Years,Hits2Years,PPG2Years,PPP2Years,PPTOI2Years,ESTOI2Years,PIM22Years,FOPct2Years,ShtPct2Years,GWG2Years,OTG2Years,SHG2Years,SHP2Years,SHTOI2Years,Blocks2Years,PlusMinus2Years,Pts22Years,Shifts2Years,TOIperGame2Years,EVTOIperGame2Years,SHTOIperGame2Years,PPTOIperGame2Years,Assists3Years,Goals3Years,Points3Years,PIMS3Years,Shots3Years,Games3Years,Hits3Years,PPG3Years,PPP3Years,PPTOI3Years,ESTOI3Years,PIM23Years,FOPct3Years,ShtPct3Years,GWG3Years,OTG3Years,SHG3Years,SHP3Years,SHTOI3Years,Blocks3Years,PlusMinus3Years,Pts23Years,Shifts3Years,TOIperGame3Years,EVTOIperGame3Years,SHTOIperGame3Years,PPTOIperGame3Years,Assists4Years,Goals4Years,Points4Years,PIMS4Years,Shots4Years,Games4Years,Hits4Years,PPG4Years,PPP4Years,PPTOI4Years,ESTOI4Years,PIM24Years,FOPct4Years,ShtPct4Years,GWG4Years,OTG4Years,SHG4Years,SHP4Years,SHTOI4Years,Blocks4Years,PlusMinus4Years,Pts24Years,Shifts4Years,TOIperGame4Years,EVTOIperGame4Years,SHTOIperGame4Years,PPTOIperGame4Years,Assists5Years,Goals5Years,Points5Years,PIMS5Years,Shots5Years,Games5Years,Hits5Years,PPG5Years,PPP5Years,PPTOI5Years,ESTOI5Years,PIM25Years,FOPct5Years,ShtPct5Years,GWG5Years,OTG5Years,SHG5Years,SHP5Years,SHTOI5Years,Blocks5Years,PlusMinus5Years,Pts25Years,Shifts5Years,TOIperGame5Years,EVTOIperGame5Years,SHTOIperGame5Years,PPTOIperGame5Years
0,34.0,C,34,CZE,73.0,190,29.0,19.0,48.0,40.0,145.0,58.0,30.0,3.0,11.0,156.22,814.54,40.0,44.85,13.10,4.0,0.0,1.0,1.0,50.44,16.0,18.0,48.0,1392.0,17.37,14.03,0.52,2.41,47.0,31.0,78.0,32.0,247.0,77.0,51.0,12.0,31.0,274.48,1011.27,32.0,29.88,12.55,6.0,1.0,2.0,5.0,143.56,17.0,18.0,78.0,1738.0,18.34,13.08,1.52,3.34,35.0,20.0,55.0,38.0,263.0,74.0,51.0,7.0,19.0,265.47,981.13,38.0,45.61,7.60,8.0,3.0,0.0,2.0,119.33,20.0,10.0,55.0,1777.0,18.28,13.15,1.36,3.35,48.0,21.0,69.0,38.0,267.0,75.0,45.0,8.0,31.0,326.47,1051.03,38.0,38.88,7.87,5.0,1.0,0.0,0.0,18.29,17.0,1.0,69.0,1716.0,18.37,14.00,0.14,4.21,29.0,16.0,45.0,20.0,142.0,38.0,22.0,6.0,21.0,175.53,516.03,20.0,20.00,11.27,3.0,0.0,0.0,0.0,13.38,4.0,11.0,45.0,854.0,18.34,13.34,0.21,4.37
1,34.0,C,31,USA,71.0,200,47.0,12.0,59.0,60.0,180.0,78.0,30.0,5.0,19.0,206.05,1190.14,60.0,50.76,6.67,1.0,0.0,0.0,0.0,158.38,34.0,1.0,59.0,2002.0,19.56,15.15,2.02,2.38,42.0,16.0,58.0,60.0,271.0,77.0,61.0,3.0,17.0,302.32,1224.37,60.0,52.36,5.90,7.0,1.0,1.0,1.0,94.39,23.0,-2.0,58.0,1977.0,21.03,15.54,1.13,3.55,54.0,16.0,70.0,36.0,242.0,81.0,48.0,7.0,29.0,347.12,1198.28,36.0,52.53,6.61,3.0,0.0,0.0,1.0,65.42,24.0,3.0,70.0,1844.0,19.53,14.47,0.48,4.17,47.0,13.0,60.0,42.0,248.0,72.0,21.0,4.0,23.0,321.45,1038.30,42.0,52.15,5.24,1.0,1.0,0.0,0.0,2.39,17.0,7.0,60.0,1601.0,18.55,14.25,0.02,4.28,51.0,33.0,84.0,42.0,244.0,82.0,20.0,9.0,33.0,416.00,1110.23,42.0,52.58,13.52,5.0,0.0,0.0,0.0,13.50,16.0,8.0,84.0,1867.0,18.46,13.32,0.10,5.04
2,35.0,R,31,USA,67.0,178,18.0,28.0,46.0,26.0,237.0,61.0,36.0,10.0,16.0,181.23,967.05,26.0,53.84,11.81,3.0,1.0,0.0,0.0,117.18,33.0,3.0,46.0,1603.0,20.45,15.51,1.55,2.58,40.0,20.0,60.0,32.0,248.0,81.0,44.0,3.0,11.0,196.24,1097.05,32.0,38.63,8.06,1.0,0.0,3.0,4.0,80.53,28.0,12.0,60.0,1714.0,16.58,13.32,0.59,2.25,31.0,22.0,53.0,46.0,257.0,82.0,52.0,8.0,21.0,285.23,1150.39,46.0,54.54,8.56,4.0,1.0,1.0,1.0,61.59,14.0,1.0,53.0,1870.0,18.16,14.01,0.45,3.28,20.0,25.0,45.0,36.0,194.0,62.0,30.0,11.0,18.0,275.29,859.04,36.0,38.71,12.89,4.0,0.0,0.0,0.0,32.33,10.0,-3.0,45.0,1397.0,18.49,13.51,0.31,4.26,41.0,48.0,89.0,46.0,291.0,82.0,47.0,24.0,34.0,410.35,1091.53,46.0,38.35,16.49,10.0,3.0,1.0,1.0,122.49,17.0,18.0,89.0,2012.

In [21]:
# conda install -c anaconda keras-gpu
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(list(CDF2.columns))
    
    
CDF2.to_csv("Pre_Model_Dataframe.csv")

['Target',
 'Position',
 'Age',
 'Country',
 'Height',
 'Weight',
 'AssistsLastYear',
 'GoalsLastYear',
 'PointsLastYear',
 'PIMSLastYear',
 'ShotsLastYear',
 'GamesLastYear',
 'HitsLastYear',
 'PPGLastYear',
 'PPPLastYear',
 'PPTOILastYear',
 'ESTOILastYear',
 'PIM2LastYear',
 'FOPctLastYear',
 'ShtPctLastYear',
 'GWGLastYear',
 'OTGLastYear',
 'SHGLastYear',
 'SHPLastYear',
 'SHTOILastYear',
 'BlocksLastYear',
 'PlusMinusLastYear',
 'Pts2LastYear',
 'ShiftsLastYear',
 'TOIperGameLastYear',
 'EVTOIperGameLastYear',
 'SHTOIperGameLastYear',
 'PPTOIperGameLastYear',
 'Assists2Years',
 'Goals2Years',
 'Points2Years',
 'PIMS2Years',
 'Shots2Years',
 'Games2Years',
 'Hits2Years',
 'PPG2Years',
 'PPP2Years',
 'PPTOI2Years',
 'ESTOI2Years',
 'PIM22Years',
 'FOPct2Years',
 'ShtPct2Years',
 'GWG2Years',
 'OTG2Years',
 'SHG2Years',
 'SHP2Years',
 'SHTOI2Years',
 'Blocks2Years',
 'PlusMinus2Years',
 'Pts22Years',
 'Shifts2Years',
 'TOIperGame2Years',
 'EVTOIperGame2Years',
 'SHTOIperGame2Years',

In [20]:
# FinalDataframe = CDF2

# from keras import Model
# from keras.layers import Dense

# from keras_pandas.Automater import Automater

# categorical_vars = ['Position','Country']
# numerical_vars = ['Age','Height','Weight','AssistsLastYear','GoalsLastYear', 'PointsLastYear', 'PIMSLastYear',
#                   'ShotsLastYear', 'GamesLastYear', 'HitsLastYear', 'PPGLastYear', 'PPPLastYear', 'PPTOILastYear',
#                   'ESTOILastYear', 'PIM2LastYear', 'FOPctLastYear', 'ShtPctLastYear', 'GWGLastYear', 'OTGLastYear',
#                   'SHGLastYear', 'SHPLastYear', 'SHTOILastYear', 'BlocksLastYear', 'PlusMinusLastYear', 'Pts2LastYear',
#                   'ShiftsLastYear', 'TOIperGameLastYear', 'EVTOIperGameLastYear', 'SHTOIperGameLastYear',
#                   'PPTOIperGameLastYear', 'Assists2Years', 'Goals2Years', 'Points2Years', 'PIMS2Years', 'Shots2Years',
#                   'Games2Years', 'Hits2Years', 'PPG2Years', 'PPP2Years', 'PPTOI2Years', 'ESTOI2Years', 'PIM22Years',
#                   'FOPct2Years', 'ShtPct2Years', 'GWG2Years', 'OTG2Years', 'SHG2Years', 'SHP2Years', 'SHTOI2Years',
#                   'Blocks2Years', 'PlusMinus2Years', 'Pts22Years', 'Shifts2Years', 'TOIperGame2Years', 'EVTOIperGame2Years',
#                   'SHTOIperGame2Years', 'PPTOIperGame2Years', 'Assists3Years', 'Goals3Years', 'Points3Years', 'PIMS3Years',
#                   'Shots3Years', 'Games3Years', 'Hits3Years', 'PPG3Years', 'PPP3Years', 'PPTOI3Years', 'ESTOI3Years',
#                   'PIM23Years', 'FOPct3Years', 'ShtPct3Years', 'GWG3Years', 'OTG3Years', 'SHG3Years', 'SHP3Years',
#                   'SHTOI3Years', 'Blocks3Years', 'PlusMinus3Years', 'Pts23Years', 'Shifts3Years', 'TOIperGame3Years',
#                   'EVTOIperGame3Years', 'SHTOIperGame3Years', 'PPTOIperGame3Years', 'Assists4Years', 'Goals4Years',
#                   'Points4Years', 'PIMS4Years', 'Shots4Years', 'Games4Years', 'Hits4Years', 'PPG4Years', 'PPP4Years',
#                   'PPTOI4Years', 'ESTOI4Years', 'PIM24Years', 'FOPct4Years', 'ShtPct4Years', 'GWG4Years', 'OTG4Years',
#                   'SHG4Years', 'SHP4Years', 'SHTOI4Years', 'Blocks4Years', 'PlusMinus4Years', 'Pts24Years', 'Shifts4Years',
#                   'TOIperGame4Years', 'EVTOIperGame4Years', 'SHTOIperGame4Years', 'PPTOIperGame4Years', 'Assists5Years',
#                   'Goals5Years', 'Points5Years', 'PIMS5Years', 'Shots5Years', 'Games5Years', 'Hits5Years', 'PPG5Years',
#                   'PPP5Years', 'PPTOI5Years', 'ESTOI5Years', 'PIM25Years', 'FOPct5Years', 'ShtPct5Years', 'GWG5Years',
#                   'OTG5Years', 'SHG5Years', 'SHP5Years', 'SHTOI5Years', 'Blocks5Years', 'PlusMinus5Years', 'Pts25Years', 
#                   'Shifts5Years', 'TOIperGame5Years', 'EVTOIperGame5Years', 'SHTOIperGame5Years', 'PPTOIperGame5Years']

# auto = Automater(categorical_vars=categorical_vars, numerical_vars=numerical_vars, text_vars=text_vars,
#  response_var='Target')
# X, y = auto.fit_transform(observations)

# x = auto.input_nub
# x = Dense(256)(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activate='relu')(x)
# x = Dense(128)(x)

# x = auto.output_nub(x)

# model = Model(inputs=auto.input_layers, outputs=x)
# model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train model
# model.fit(X, y, epochs=4, validation_split=.2)

ModuleNotFoundError: No module named 'keras_pandas'

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.models import load_model
# from keras.layers import Dropout
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.constraints import maxnorm
# from keras.optimizers import SGD
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split
# from pandas_profiling import ProfileReport
# from plotnine import *
# import keras
# import argparse
# import numpy as np
# import tensorflow as tf
# import h5py
# import pandas as pd
# import pyreadr
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
# from sklearn.metrics import r2_score
# import rpy2.robjects as ro
# from rpy2.robjects import pandas2ri
# from rpy2.robjects.conversion import localconverter
# from keras.callbacks import EarlyStopping
# from keras.callbacks import ModelCheckpoint
# from keras.callbacks import LearningRateScheduler
# from keras.callbacks import TensorBoard
# from keras import layers
# from plotnine import *
# import statistics
# from scipy.stats import ttest_ind
# import scipy as sp
# pandas2ri.activate()
# # import csv

In [ ]:
# pip install pandas_profiling


In [26]:


Goals = pd.read_hdf("Goals.h5", key = 'Goals')
Hits = pd.read_hdf("Hits.h5", key = "Hits")
Missed_Shots = pd.read_hdf("Missed.h5", key = "Missed_Shots")
Shots = pd.read_hdf("Shots.h5", key = "Shots")
Penalties = pd.read_hdf("Penalties.h5", key = "Penalties")

SeasonDetails = pd.read_hdf("AllSeasonDetails.h5", key = "SeasonDetails")
SkaterStatistics = pd.read_hdf("SkaterStats.h5", key = "SkaterStatistics")
GoalieStatistics = pd.read_hdf("GoalieStats.h5", key = "GoalieStatistics")
FullRoster = pd.read_hdf("NHLRoster.h5", key = "FullRoster")
PlayerDetails = pd.read_hdf("PlayerDetails.h5", key = "PlayerDetails")


Goals.to_csv("Goals.csv")
Hits.to_csv("Hits.csv")
Missed_Shots.to_csv("Missed.csv")
Shots.to_csv("Shots.csv")
Penalties.to_csv("Penalties.csv")

SeasonDetails.to_csv("AllSeasonDetails.csv")
SkaterStatistics.to_csv("SkaterStats.csv")
GoalieStatistics.to_csv("GoalieStats.csv")
FullRoster.to_csv("NHLRoster.csv")
PlayerDetails.to_csv("PlayerDetails.csv")